In [1]:
import pandas as pd
from konlpy.tag import Okt

In [2]:
okt = Okt()

In [3]:
# 긍정 점수 사전 읽기
positive_dict = pd.read_csv('positive_score_dict.csv')
positive_dict = positive_dict.drop_duplicates(subset='단어')
positive_dict.set_index('단어', inplace=True)

In [4]:
result2 = pd.read_csv('community_token_final.csv', low_memory=False)

In [5]:
# 각 열에 대해 okt.morphs(stem=True) 적용 및 문자열 변환
result2['형용사_어간'] = result2['내용_형용사_str'].apply(
    lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['부사_어간'] = result2['내용_부사_str'].apply(
    lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['동사_어간'] = result2['내용_동사_str'].apply(
    lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

In [ ]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = [0] * len(result2)  # 초기화

for idx in range(len(result2)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_명사_str', '형용사_어간', '부사_어간', '동사_어간']:
        word_list = result2.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list) or word_list.strip() == "":  # Null 또는 빈 문자열 처리
            print(f"행 {idx}, 열 {column}은 결측값 또는 빈 문자열입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(', ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            else:
                print(f"  단어: {word}는 사전에 없음")

    row_scores[idx] = row_score  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    result2['긍정점수'] = [1 if score > 0 else -1 if score < 0 else 0 for score in row_scores]

except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(result2)}, row_scores 길이: {len(row_scores)}")

# 최종 점수 출력
print("\n행별 긍정점수:")
print(result2[['긍정점수']])